In [35]:
from datasets import load_dataset
import numpy as np
from transformers import (
    TrainingArguments, 
    Trainer,
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    pipeline
)   
from pprint import pprint
from torchinfo import summary
import tqdm
import datasets
import evaluate
import json

metric = evaluate.load("glue", "mrpc")

In [2]:
# raw_datasets = load_dataset("amazon_polarity")
raw_datasets = load_dataset("glue","sst2")
raw_datasets


Found cached dataset glue (/home/azureuser/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [3]:
dir(raw_datasets['train'])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_iter_shards',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',
 '_select_contiguous',
 '_select_with_indices_mapping',
 '_split',
 'add_column',
 'add_elasticsearch_ind

In [4]:
type(raw_datasets['train'])

datasets.arrow_dataset.Dataset

In [5]:
raw_datasets['validation'][0:5]

{'sentence': ["it 's a charming and often affecting journey . ",
  'unflinchingly bleak and desperate ',
  'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . ',
  "the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . ",
  "it 's slow -- very , very slow . "],
 'label': [1, 0, 1, 1, 0],
 'idx': [0, 1, 2, 3, 4]}

In [6]:
raw_datasets['test'][0:5]

{'sentence': ['uneasy mishmash of styles and genres .',
  "this film 's relationship to actual tension is the same as what christmas-tree flocking in a spray can is to actual snow : a poor -- if durable -- imitation .",
  'by the end of no such thing the audience , like beatrice , has a watchful affection for the monster .',
  'director rob marshall went out gunning to make a great one .',
  'lathan and diggs have considerable personal charm , and their screen rapport makes the old story seem new .'],
 'label': [-1, -1, -1, -1, -1],
 'idx': [0, 1, 2, 3, 4]}

In [7]:
raw_datasets['train'].shape

(67349, 3)

In [8]:
raw_datasets['train'][5000:5003]

{'sentence': ['entirely stale concept ',
  'will amuse or entertain them ',
  'wobbly premise work '],
 'label': [0, 1, 0],
 'idx': [5000, 5001, 5002]}

In [9]:
raw_datasets['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [10]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [11]:
tokenized_sentences = tokenizer(raw_datasets['train'][0:3]['sentence'])
pprint(tokenized_sentences)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102],
               [101,
                3397,
                2053,
                15966,
                1010,
                2069,
                4450,
                2098,
                18201,
                2015,
                102],
               [101,
                2008,
                7459,
                2049,
                3494,
                1998,
                10639,
                2015,
                2242,
                2738,
                3376,
                2055,
                2529,
                3267,
                102]]}


In [12]:
def tokenize_fn(batch):
    return tokenizer(batch['sentence'], truncation=True)


In [13]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)

Loading cached processed dataset at /home/azureuser/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-b3703a31712e058e.arrow
Loading cached processed dataset at /home/azureuser/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ff6d75692409d3ef.arrow
Loading cached processed dataset at /home/azureuser/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7cff44692f0b7ecb.arrow


In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [15]:
tokenized_datasets['validation'][0:5]['label']

[1, 0, 1, 1, 0]

In [16]:
training_args = TrainingArguments(
    'my_trainer',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=1
)

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels = 2
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [18]:
type(model)

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [19]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [20]:
summary(model)

Layer (type:depth-idx)                                  Param #
DistilBertForSequenceClassification                     --
├─DistilBertModel: 1-1                                  --
│    └─Embeddings: 2-1                                  --
│    │    └─Embedding: 3-1                              23,440,896
│    │    └─Embedding: 3-2                              393,216
│    │    └─LayerNorm: 3-3                              1,536
│    │    └─Dropout: 3-4                                --
│    └─Transformer: 2-2                                 --
│    │    └─ModuleList: 3-5                             42,527,232
├─Linear: 1-2                                           590,592
├─Linear: 1-3                                           1,538
├─Dropout: 1-4                                          --
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0

In [21]:
params_before = []
for name, p in model.named_parameters():
    params_before.append(p.detach().cpu().numpy())

In [22]:
metric = evaluate.load("glue", "mrpc")

In [23]:
metric.compute(predictions = [1,0,1], references = [1,0,0])

{'accuracy': 0.6666666666666666, 'f1': 0.6666666666666666}

In [24]:
def compute_metrics(logit_and_labels):
    logits, labels = logit_and_labels
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, reference=labels)

In [25]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics
)

In [26]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/anaconda/envs/transformer-gpu-env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 67349
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8419
  Number of trainable parameters = 66955010


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.200800,0.348379


Attempted to log scalar metric loss:
0.4032
Attempted to log scalar metric learning_rate:
4.7030526190759e-05
Attempted to log scalar metric epoch:
0.06
Attempted to log scalar metric loss:
0.3588
Attempted to log scalar metric learning_rate:
4.4061052381518e-05
Attempted to log scalar metric epoch:
0.12
Attempted to log scalar metric loss:
0.3109
Attempted to log scalar metric learning_rate:
4.109157857227699e-05
Attempted to log scalar metric epoch:
0.18
Attempted to log scalar metric loss:
0.3177
Attempted to log scalar metric learning_rate:
3.812210476303599e-05
Attempted to log scalar metric epoch:
0.24
Attempted to log scalar metric loss:
0.3032
Attempted to log scalar metric learning_rate:
3.515263095379499e-05
Attempted to log scalar metric epoch:
0.3
Attempted to log scalar metric loss:
0.2805
Attempted to log scalar metric learning_rate:
3.218315714455399e-05
Attempted to log scalar metric epoch:
0.36
Attempted to log scalar metric loss:
0.2913
Attempted to log scalar metric 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 8


Attempted to log scalar metric eval_loss:
0.34837907552719116
Attempted to log scalar metric eval_runtime:
3.1223
Attempted to log scalar metric eval_samples_per_second:
279.283
Attempted to log scalar metric eval_steps_per_second:
34.91
Attempted to log scalar metric epoch:
1.0


Saving model checkpoint to my_trainer/checkpoint-8419
Configuration saved in my_trainer/checkpoint-8419/config.json
Model weights saved in my_trainer/checkpoint-8419/pytorch_model.bin
tokenizer config file saved in my_trainer/checkpoint-8419/tokenizer_config.json
Special tokens file saved in my_trainer/checkpoint-8419/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




Attempted to log scalar metric train_runtime:
889.9193
Attempted to log scalar metric train_samples_per_second:
75.68
Attempted to log scalar metric train_steps_per_second:
9.46
Attempted to log scalar metric total_flos:
518400815624736.0
Attempted to log scalar metric train_loss:
0.2661926889379909
Attempted to log scalar metric epoch:
1.0


TrainOutput(global_step=8419, training_loss=0.2661926889379909, metrics={'train_runtime': 889.9193, 'train_samples_per_second': 75.68, 'train_steps_per_second': 9.46, 'total_flos': 518400815624736.0, 'train_loss': 0.2661926889379909, 'epoch': 1.0})

In [27]:
trainer.save_model('my_saved_model')

Saving model checkpoint to my_saved_model
Configuration saved in my_saved_model/config.json
Model weights saved in my_saved_model/pytorch_model.bin
tokenizer config file saved in my_saved_model/tokenizer_config.json
Special tokens file saved in my_saved_model/special_tokens_map.json


In [28]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
'1. Models & Tokenizers.ipynb'		    my_saved_model
'2. Fine-tuning Sentiment Analysis.ipynb'   my_trainer


In [41]:
newmodel = pipeline('text-classification', model = 'my_saved_model', device = 0)

loading configuration file my_saved_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "label2id": null,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading configuration file my_saved_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention

In [42]:
newmodel('This movie is great!')

[{'label': 'positive', 'score': 0.9994148015975952}]

In [43]:
newmodel('This movie is bad')

[{'label': 'negative', 'score': 0.9976747632026672}]

In [44]:
newmodel('This movie is not good')

[{'label': 'negative', 'score': 0.9969557523727417}]

In [36]:
!cat my_saved_model/config.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}


In [38]:
config_path = 'my_saved_model/config.json'
with open(config_path) as f:
    j = json.load(f)
j['id2label'] = {0:'negative', 1: 'positive'}

with open(config_path, 'w') as f:
    json.dump(j, f, indent=2)

In [39]:
!cat my_saved_model/config.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
{
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522,
  "id2label": {
    "0": "negative",
    "1": "positive"
  }
}

In [45]:
newmodel = pipeline('text-classification', model = 'my_saved_model', device = 0)

loading configuration file my_saved_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "label2id": null,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading configuration file my_saved_model/config.json
Model config DistilBertConfig {
  "_name_or_path": "my_saved_model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention

In [46]:
newmodel('This movie is not bad')

[{'label': 'positive', 'score': 0.9970710277557373}]

In [47]:
params_after = []
for name, p in model.named_parameters():
    params_after.append(p.detach().cpu().numpy())

In [48]:
for p1, p2 in zip(params_before, params_after):
    print(np.sum(np.abs(p1-p2)))

13333.5
91.96128
1.7332611
1.1682153
1302.7913
1.6948868
1299.2146
0.0026270724
1193.4305
1.1234747
1134.9882
0.8862737
1.6528178
0.8772323
4888.768
5.6523566
4511.2227
0.72401726
1.5393064
0.71121395
1257.303
1.5205218
1262.4834
0.002978927
1108.9269
0.84454036
1062.9504
0.739426
1.5157192
0.73383045
4826.9077
5.4167795
4434.05
0.7533361
1.5804412
0.74911803
1270.4647
1.4963179
1271.8312
0.0024332847
1106.8967
0.7831686
1102.844
0.77894014
1.5596521
0.8338433
4853.8276
5.4934216
4316.471
0.68979156
1.3808801
0.6435543
1248.2577
1.3866069
1264.1915
0.0026707123
1117.9866
0.7406681
1062.1107
0.70871115
1.3610401
0.7217765
4721.2993
5.4737387
4083.3606
0.72429836
1.2973986
0.7309301
1194.5912
1.4834925
1211.0432
0.0017280958
1010.7979
0.7786219
1022.9903
0.93903685
1.3283542
0.9510894
4499.1655
5.4275646
3672.497
0.8407453
1.2950855
0.78732467
1155.5338
1.3688072
1148.0323
0.0013841816
957.8791
0.79997224
958.9434
1.014788
1.2900279
1.1735375
3779.9058
4.836296
3375.7666
0.99907655
1.447